# Generate MEIs for one Dataset

In [1]:
%load_ext autoreload
%autoreload 2

import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 200

name = 'iclr' #"test"
dj.config['schema_name'] = f"konstantin_nnsysident_{name}"

#schema = dj.schema("nnfabrik_color_mei")

import nnfabrik
from nnfabrik.main import *
from nnfabrik import builder #, main

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nnsysident.tables.experiments import *
from nnsysident.tables.mei import *
from nnsysident.utility.measures import get_correlations

fetch_download_path = './mei_downloads'

Connecting konstantin@134.2.168.16:3306


In [2]:
def best_n_unit_ids(model, dataloaders, n, device='cuda'):
    test_correlation = get_correlations(model, dataloaders["test"], device=device, as_dict=False, per_neuron=True)
    indices = np.argsort(test_correlation)
    indices = indices[-n:]
    
    for i, (data_key, dataloader) in enumerate(dataloaders['test'].items()):
        assert i>=0, 'More than one dataset not supported (yet)'
        unit_ids = dataloader.dataset.neurons.unit_ids[indices]
    return unit_ids, indices

___

In [ ]:
def corrs(KEY, transfer, seed=1, device='cuda'): 
    if transfer:
        dataloaders, model = (TrainedModelTransfer() & KEY & 'seed={}'.format(seed)).load_model()
    else:
        dataloaders, model = (TrainedModel() & KEY & 'seed={}'.format(seed)).load_model()
    test_correlation = get_correlations(model, dataloaders["test"], device=device, as_dict=False, per_neuron=True)
    return test_correlation

In [ ]:
key = (ExperimentsTransfer.Restrictions() & 'experiment_name = "Real, core_transfer (animal), Core By Seed, se2d_fullgaussian2d, 11-S -> 20457-5-9"') & 'dataset_hash = "71c9ac7a98e066544ad88eb47ea282ec"'
test_correlation_transfer = corrs(key, True)

In [ ]:
key = (Experiments.Restrictions() & 'experiment_name = "Real, Direct, se2d_fullgaussian2d, 20457-5-9"') & 'dataset_hash = "71c9ac7a98e066544ad88eb47ea282ec"'
test_correlation = corrs(key, False)

In [ ]:
n = 10
diff = test_correlation_transfer - test_correlation
indices = np.argsort(diff)
indices = indices[-n:]
for i, (data_key, dataloader) in enumerate(dataloaders['test'].items()):
    assert i>=0, 'More than one dataset not supported (yet)'
    unit_ids = dataloader.dataset.neurons.unit_ids[indices]

In [ ]:
unit_ids

In [ ]:
test_correlation[indices]

In [ ]:
test_correlation_transfer[indices]

# Select Dataset

In [3]:
key = (Experiments.Restrictions() & 'experiment_name = "Real, Direct, se2d_fullgaussian2d, 20457-5-9"') & 'dataset_hash = "71c9ac7a98e066544ad88eb47ea282ec"'


In [4]:
key

experiment_name name of experiment,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,model_fn name of the model function,model_hash hash of the model configuration,experiment_restriction_ts UTZ timestamp at time of insertion
"Real, Direct, se2d_fullgaussian2d, 20457-5-9",nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,2020-09-01 14:55:16


# Creating the ensemble

#### the key has to restrict the TrainedModel to all the models that should be an ensemble. The dataset hash MUST be unique

In [5]:
(TrainedModel() & key)

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,comment short description,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,1,"Bayesian optimization of Hyper params..se2d_fullgaussian2d, neuron_n=5335, image_n=4472.neuron_n=5335, image_n=4472",0.293081,=BLOB=,kklurz,2020-09-01 19:40:52
nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,2,"Bayesian optimization of Hyper params..se2d_fullgaussian2d, neuron_n=5335, image_n=4472.neuron_n=5335, image_n=4472",0.297961,=BLOB=,kklurz,2020-09-01 19:32:15
nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,3,"Bayesian optimization of Hyper params..se2d_fullgaussian2d, neuron_n=5335, image_n=4472.neuron_n=5335, image_n=4472",0.296941,=BLOB=,kklurz,2020-09-02 09:46:22
nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,4,"Bayesian optimization of Hyper params..se2d_fullgaussian2d, neuron_n=5335, image_n=4472.neuron_n=5335, image_n=4472",0.295713,=BLOB=,kklurz,2020-09-01 19:36:02
nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,5,"Bayesian optimization of Hyper params..se2d_fullgaussian2d, neuron_n=5335, image_n=4472.neuron_n=5335, image_n=4472",0.297428,=BLOB=,kklurz,2020-09-01 19:39:51


# Create an ensemble

In [6]:
TrainedEnsembleModel().create_ensemble(key=key, comment='Real, Direct, se2d_fullgaussian2d, 20457-5-9')

DuplicateError: ("Duplicate entry 'nnsysident.datasets.mouse_loaders.static_loaders-71c9ac7a98e0...' for key 'PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [ ]:
TrainedEnsembleModel & key

In [7]:
# Check whether the ensemble has the correct amount of members:
TrainedEnsembleModel.Member() & key

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,ensemble_hash the hash of the ensemble,model_fn name of the model function,model_hash hash of the model configuration,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder
nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,4dc2b15a95c86f907b7417d8811f54fe,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,1
nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,4dc2b15a95c86f907b7417d8811f54fe,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,2
nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,4dc2b15a95c86f907b7417d8811f54fe,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,3
nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,4dc2b15a95c86f907b7417d8811f54fe,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,4
nnsysident.datasets.mouse_loaders.static_loaders,71c9ac7a98e066544ad88eb47ea282ec,4dc2b15a95c86f907b7417d8811f54fe,nnsysident.models.models.se2d_fullgaussian2d,befddf46eb23b8ee32f0fbb67d490baf,nnsysident.training.trainers.standard_trainer,d41d8cd98f00b204e9800998ecf8427e,5


# Populating the MEISelector table, to match the unit IDs and the units within the model

In [ ]:
MEISelector().populate(key, display_progress=True)

In [ ]:
# one entry in the table corresponds to one neuron. 
#he unit_id comes straight from the dataset, the unit index is the position of that unit in the model.
MEISelector() & key

# Selecting the MEIMethod

#### The default here is for normalized images

# for Normalized (z-scored) Images

In [ ]:
# it's using the function gradient_ascent from the mei package.
# as the default processing steps during generation, we'll constrain the L2norm or std of the MEI (basically a contrast constraint)
# also a glaussian blurring on the gradient is applied, smoothing the image, getting rid of high frequency artefacts.


method_fn = 'mei.methods.gradient_ascent'

method_config = {'initial': {'path': 'mei.initial.RandomNormal'},
                 'optimizer': {'path': 'torch.optim.SGD', 'kwargs': {'lr': 2.0}},
                 'precondition': {'path': 'mei.legacy.ops.GaussianBlur',
                 'kwargs': {'sigma': 1}},
                 'postprocessing': {'path': 'mei.legacy.ops.ChangeNorm',
                 'kwargs': {'norm': 15}},
                 'stopper': {'path': 'mei.stoppers.NumIterations',
                 'kwargs': {'num_iterations': 500}},
                 'objectives': [{'path': 'mei.objectives.EvaluationObjective',
                 'kwargs': {'interval': 10}}],
                 'device': 'cuda'}

MEIMethod().add_method(method_fn, method_config, "normalized image, norm=15, lr=2, iter=500")

# 8bit Images

In [ ]:
# it's using the function gradient_ascent from the mei package.
# as the default processing steps during generation, we'll constrain the L2norm or std of the MEI (basically a contrast constraint)
# also a glaussian blurring on the gradient is applied, smoothing the image, getting rid of high frequency artefacts.

# method_fn = 'mei.methods.gradient_ascent'

# sigma = 1
# lr = 1e4
# std = 5
# n_iters=500

# method_config = {'initial': {'path': 'nnsysident.meis.initial.CustomRandomNormal'},
#                  'optimizer': {'path': 'torch.optim.SGD', 'kwargs': {'lr': lr}},
#                  'precondition': {'path': 'mei.legacy.ops.GaussianBlur',
#                  'kwargs': {'sigma': sigma}},
#                  'postprocessing': {'path': 'mei.legacy.ops.ChangeStd',
#                  'kwargs': {'std': std, 'zero_mean': False}},
#                  'stopper': {'path': 'mei.stoppers.NumIterations',
#                  'kwargs': {'num_iterations': n_iters}},
#                  'objectives': [{'path': 'mei.objectives.EvaluationObjective',
#                  'kwargs': {'interval': 10}}],
#                  'device': 'cuda'}

# MEIMethod().add_method(method_fn, method_config, f"8 bit image, std={std}, lr={lr}, iter={n_iters}")



#############################################################
method_fn = 'mei.methods.gradient_ascent'
sigma = 1
lr = 1e4
std = 2
mean= 111.3
n_iters=500
method_config = {
   'initial': {'path': 'nnsysident.meis.initial.CustomRandomNormal'},
   'optimizer': {'path': 'torch.optim.SGD', 
                 'kwargs': {'lr': lr}},
   'precondition': {'path': 'mei.legacy.ops.GaussianBlur',
                    'kwargs': {'sigma': sigma}},
   'postprocessing': {'path': 'nnsysident.meis.ops.ChangeStdClampedMean',
                      'kwargs': {'std': std, 
                                 'x_min': 0, 
                                 'x_max': 255, 
                                 'clamped_mean': mean}},
   'stopper': {'path': 'mei.stoppers.NumIterations',
               'kwargs': {'num_iterations': n_iters}},
   'objectives': [{'path': 'mei.objectives.EvaluationObjective',
                   'kwargs': {'interval': 10}}],
   'device': 'cuda'
}
MEIMethod().add_method(method_fn, method_config, f"8 bit image, std={std}, mean={mean}, lr={lr}, iter={n_iters}")

In [ ]:
MEIMethod()   #.fetch1('method_config')

# Populate the MEI table

In [18]:
seed = 1
dataloaders, model = (TrainedModel() & key & 'seed={}'.format(seed)).load_model()

data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip


In [37]:
dataloaders['train']['20457-5-9-0'].dataset.neurons.unit_ids

array([2815, 3982, 2002, ..., 1377,  101, 2552], dtype=uint16)

In [38]:
n = 20

dataset_hash = '71c9ac7a98e066544ad88eb47ea282ec'
ensemble_hash = '4dc2b15a95c86f907b7417d8811f54fe'
method_hash='8db856e30d03df10c2c41326ff7e5422'
#unit_ids, indices = best_n_unit_ids(model, dataloaders, n)
#unit_ids = np.array([ 259, 1641, 1369, 2999, 2532, 1951, 5443, 3648, 3316, 1038])
unit_ids = dataloaders['train']['20457-5-9-0'].dataset.neurons.unit_ids

mei_seed=1

mei_restriction = dj.AndList(['dataset_hash = "{}"'.format(dataset_hash), 
             'ensemble_hash = "{}"'.format(ensemble_hash), 
             'method_hash = "{}"'.format(method_hash), 
             'unit_id in {}'.format(tuple(unit_ids)),
             'mei_seed = {}'.format(mei_seed)])

MEI.progress(mei_restriction)

MEI                  Completed 30 of 5335 (0.6%)   2020-11-15 19:01:13


(5305, 5335)

In [39]:
MEI.populate(mei_restriction,
             display_progress=True, 
             order='random',
             reserve_jobs=True)

  0%|          | 0/5305 [00:00<?, ?it/s]

data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip
data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip
data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip
data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip
data/static20457-5-9-preproc0 exists already. Not unpacking data/static20457-5-9-preproc0.zip


 27%|██▋       | 1442/5305 [10:59:23<29:26:28, 27.44s/it]


RequestTimeout: RequestTimeout: message: Your socket connection to the server was not read from or written to within the timeout period.

# Fetch MEIs and plot activations

In [30]:
# fetch the meis and outputs
mei_paths, output_paths, unit_IDs, score = (MEISelector * MEI() & mei_restriction).fetch("mei", "output","unit_id","score", download_path=fetch_download_path)

In [31]:
MEIs = np.stack([torch.load(path).detach().cpu().numpy().squeeze() for path in mei_paths])
evaluations = np.stack([torch.load(path)['mei.objectives.EvaluationObjective']["values"] for path in output_paths])
t_evaluations = np.stack([torch.load(path)['mei.objectives.EvaluationObjective']["times"] for path in output_paths])

In [33]:
mei_direct = dict(unit_IDs = unit_IDs, MEIs = MEIs)
torch.save(mei_direct, 'mei_direct_highestdiff')

# Plot Activations

In [ ]:
plt.plot(t_evaluations.T, evaluations.T)
sns.despine(trim=True, offset=10)
plt.xlabel("iterations")
plt.ylabel("activation")
plt.title("Activations over iterations for all neurons");

# plot MEIs

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(10,5), dpi=150)
for i, ax in enumerate(axes.flatten()):
    ax.imshow(MEIs[i], cmap="gray", vmin=MEIs.min(), vmax=MEIs.max() ) # vmin=MEIs.min(), vmax=MEIs.max() 
    ax.axis("off")
    ax.text(0.5, 2.5, str(unit_IDs[i]), va='center', fontsize=8.5, color='k')
fig.savefig('mei.png')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(10,5), dpi=150)
ax[0].hist(MEIs.flatten(), bins=20)
ax[0].set_xlabel('pixel value')
ax[0].set_title('Pixel distribution all MEIs')

ax[1].hist(MEIs.max(axis=1).max(axis=1), bins=20)
ax[1].set_xlabel('pixel value')
ax[1].set_title('Max pixel value per MEI')

ax[2].hist(MEIs.min(axis=1).min(axis=1), bins=20)
ax[2].set_xlabel('pixel value')
ax[2].set_title('Min pixel value per MEI')

sns.despine(trim=True)
#fig.savefig('MEI pixel distribution')